# HỒI QUY SOFTMAX (MULTINOMIAL LOGISTIC REGRESSION)

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split
from scipy import sparse
from sklearn.preprocessing import label_binarize

## Hàm softmax 

In [32]:
def convert_labels(y, C):
    Y = sparse.coo_matrix((np.ones_like(y), (y, np.arange(len(y)))), shape=(C, len(y))).toarray()
    return Y

def softmax_stable(Z):
    e_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return e_Z / e_Z.sum(axis=0)

def softmax(Z):
    e_Z = np.exp(Z)
    return e_Z / e_Z.sum(axis=0)

def softmax_regression(X, y, W_init, eta=0.05, tol=1e-4, max_count=10000):
    W = [W_init]
    C = W_init.shape[1]
    Y = convert_labels(y, C)
    N = X.shape[1]
    d = X.shape[0]

    count = 0
    check_w_after = 1500
    while count < max_count:
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = X[:, i].reshape(d, 1)
            yi = Y[:, i].reshape(C, 1)
            ai = softmax(np.dot(W[-1].T, xi))
            W_new = W[-1] + eta * xi.dot((yi - ai).T)
            count += 1
            if count % check_w_after == 0:
                if np.linalg.norm(W_new - W[-check_w_after]) < tol:
                    return W
            W.append(W_new)
    return W

In [33]:
# dự đoán và đánh giá các chỉ số 
def pred(W, X):
    A = softmax_stable(W.T.dot(X))
    return np.argmax(A, axis=0)

def evaluate_model(y_true, y_pred):
    print(classification_report(y_true, y_pred))
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred))
    auc = roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(y_pred), average='macro', multi_class='ovr')
    print(f"AUC (macro average): {auc:.4f}")

## DỮ LIỆU GỐC 

In [34]:
# Load dữ liệu
df = pd.read_csv("../../data/data_processed/data_processed.csv")

# Tách đặc trưng và nhãn
X = df.drop(columns=["NSP"]).values
y = df["NSP"].values - 1  # Chuyển về 0,1,2

# Chuẩn hóa
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Thêm bias
X_scaled = np.hstack([np.ones((X_scaled.shape[0], 1)), X_scaled])
C = len(np.unique(y))

In [35]:
# Hàm huấn luyện và đánh giá với tỉ lệ cho trước
def train_and_evaluate(X, y, test_size):
    print(f"\n--- Tỉ lệ train:test = {int((1-test_size)*10)}:{int(test_size*10)} ---")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=42)
    X_train_T = X_train.T
    X_test_T = X_test.T
    d = X_train_T.shape[0]

    W_init = np.random.randn(d, C)
    W = softmax_regression(X_train_T, y_train, W_init)[-1]

    y_pred = pred(W, X_test_T)
    evaluate_model(y_test, y_pred)

# Chạy với các tỉ lệ
for test_size in [0.2, 0.3, 0.4]:
    train_and_evaluate(X_scaled, y, test_size)



--- Tỉ lệ train:test = 8:2 ---
              precision    recall  f1-score   support

         0.0       0.92      0.94      0.93       332
         1.0       0.58      0.59      0.59        59
         2.0       0.82      0.66      0.73        35

    accuracy                           0.87       426
   macro avg       0.78      0.73      0.75       426
weighted avg       0.87      0.87      0.87       426

Confusion Matrix:
[[312  17   3]
 [ 22  35   2]
 [  4   8  23]]
AUC (macro average): 0.8054

--- Tỉ lệ train:test = 7:3 ---
              precision    recall  f1-score   support

         0.0       0.95      0.91      0.93       497
         1.0       0.50      0.72      0.59        88
         2.0       0.83      0.55      0.66        53

    accuracy                           0.86       638
   macro avg       0.76      0.73      0.73       638
weighted avg       0.88      0.86      0.86       638

Confusion Matrix:
[[454  40   3]
 [ 22  63   3]
 [  2  22  29]]
AUC (macro average

### Đánh giá tổng quan theo độ chính xác và AUC

| Tỉ lệ train:test | Accuracy | AUC (macro) | Macro F1-score |
|------------------|----------|-------------|----------------|
| 8:2              | 0.8779   | 0.9568      | 0.7665         |
| 7:3              | 0.8840   | 0.9545      | 0.7609         |
| 6:4              | **0.8931** | **0.9631**    | **0.7898**       |

**Lớp 0 (bình thường):**
Precision và Recall đều rất cao ở cả 3 tỉ lệ (trên 92%).

Đây là lớp chiếm đa số nên mô hình học tốt nhất.

**Lớp 1 (nghi ngờ):**
Hiệu suất biến động rõ rệt, Precision/Recall dao động trong khoảng 59–68%.

Tỉ lệ 6:4 cho kết quả ổn định hơn về F1 (0.63), dù Recall hơi giảm nhẹ.

**Lớp 2 (bất thường):**
Nhạy cảm với sự thay đổi tỉ lệ chia.

Tỉ lệ 6:4 có Precision 85% và F1-score gần 0.79 — tốt nhất trong 3 lựa chọn.

Lớp 1 (suspect) thường có precision và recall thấp do đặc điểm đặc trưng của nó nằm giữa hai lớp còn lại (normal và pathologic), khiến mô hình dễ nhầm lẫn. Đây là lớp trung gian về mặt y học, nên các đặc trưng không phân tách rõ ràng, dẫn đến việc mô hình thường dự đoán sai sang lớp 0 hoặc lớp 2. Đồng thời, softmax regression là mô hình tuyến tính nên càng gặp khó khăn khi ranh giới giữa các lớp không rõ ràng hoặc có sự chồng lấn trong không gian đặc trưng.

## DỮ LIỆU GIẢM CHIỀU 

### GIẢM TRƯỚC CHIA SAU 

### PCA 

In [36]:
pca_df = pd.read_csv("../../data/dimension_reduction/pca/pca_all.csv")
pca_X = pca_df.drop(columns=["NSP"]).values
pca_y = pca_df["NSP"].values - 1
scaler = StandardScaler()
pca_X_scaled = scaler.fit_transform(pca_X)

print("--- Đánh giá với PCA ---")
for test_size in [0.2, 0.3, 0.4]:
    train_and_evaluate(pca_X_scaled, pca_y, test_size)

--- Đánh giá với PCA ---

--- Tỉ lệ train:test = 8:2 ---
              precision    recall  f1-score   support

         0.0       0.98      0.78      0.87       332
         1.0       0.44      0.88      0.58        59
         2.0       0.43      0.51      0.47        35

    accuracy                           0.77       426
   macro avg       0.62      0.73      0.64       426
weighted avg       0.86      0.77      0.80       426

Confusion Matrix:
[[260  51  21]
 [  4  52   3]
 [  1  16  18]]
AUC (macro average): 0.8136

--- Tỉ lệ train:test = 7:3 ---
              precision    recall  f1-score   support

         0.0       0.97      0.79      0.87       497
         1.0       0.45      0.78      0.57        88
         2.0       0.39      0.57      0.47        53

    accuracy                           0.77       638
   macro avg       0.60      0.71      0.64       638
weighted avg       0.85      0.77      0.80       638

Confusion Matrix:
[[395  61  41]
 [ 14  69   5]
 [  0  23

### Kết quả đánh giá với PCA

| Tỉ lệ chia | Accuracy | AUC (macro) | F1-score (macro) |
|------------|----------|-------------|------------------|
| 8:2        | 0.7606   | 0.8786      | 0.6042           |
| 7:3        | **0.7759**   | 0.8740      | 0.6356           |
| 6:4        | 0.7591   | **0.9192**  | **0.6380**       |

### LDA 

In [37]:
lda_df = pd.read_csv("../../data/dimension_reduction/lda/lda_all.csv")

lda_X = lda_df.drop(columns=["NSP"]).values
lda_y = lda_df["NSP"].values - 1
lda_X_scaled = scaler.fit_transform(lda_X)

print("\n--- Đánh giá với LDA ---")
for test_size in [0.2, 0.3, 0.4]:
  train_and_evaluate(lda_X_scaled, lda_y, test_size)


--- Đánh giá với LDA ---

--- Tỉ lệ train:test = 8:2 ---
              precision    recall  f1-score   support

         0.0       0.98      0.77      0.86       332
         1.0       0.47      0.80      0.59        59
         2.0       0.37      0.66      0.47        35

    accuracy                           0.77       426
   macro avg       0.60      0.74      0.64       426
weighted avg       0.86      0.77      0.79       426

Confusion Matrix:
[[257  42  33]
 [  6  47   6]
 [  0  12  23]]
AUC (macro average): 0.8195

--- Tỉ lệ train:test = 7:3 ---
              precision    recall  f1-score   support

         0.0       0.99      0.76      0.86       497
         1.0       0.44      0.92      0.59        88
         2.0       0.42      0.57      0.48        53

    accuracy                           0.77       638
   macro avg       0.62      0.75      0.65       638
weighted avg       0.87      0.77      0.79       638

Confusion Matrix:
[[378  82  37]
 [  3  81   4]
 [  0  2

### 🔹 Kết quả đánh giá với LDA

| Tỉ lệ chia | Accuracy | AUC (macro) | F1-score (macro) |
|------------|----------|-------------|------------------|
| 8:2        | 0.7700   | 0.9112      | 0.6445           |
| 7:3        | 0.7633   | 0.8996      | 0.6356           |
| 6:4        | **0.7779** | **0.9294** | **0.6613**       |

### So sánh giữa PCA và LDA:
PCA: Đạt kết quả tốt nhất cho lớp 0, nhưng với các lớp 1 và 2, precision và recall thấp, có thể vì PCA chủ yếu tối ưu hóa việc giảm chiều mà không chú ý đến sự phân biệt giữa các lớp. AUC có phần thấp hơn so với LDA.

LDA: Mặc dù precision của lớp 1 và 2 vẫn không quá cao, nhưng recall và AUC cho thấy LDA có khả năng phân biệt các lớp tốt hơn, đặc biệt trong các tỉ lệ chia nhỏ hơn. LDA có xu hướng cải thiện khả năng phân loại cho các lớp nhỏ hơn (lớp 1 và 2), trong khi PCA vẫn tập trung vào lớp 0.

## CHIA TRƯỚC GIẢM SAU 

In [38]:
def train_test_from_file(train_path, test_path, test_ratio_label):
    print(f"\n--- Tỉ lệ train:test = {test_ratio_label} ---")
    
    # Đọc dữ liệu
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)

    X_train = train_df.drop(columns=['Unnamed: 0', "NSP"], axis=1).values.T
    y_train = train_df['NSP'].values - 1
    X_test = test_df.drop(columns=['Unnamed: 0', "NSP"], axis=1).values.T
    y_test = test_df['NSP'].values - 1

    # Huấn luyện
    d = X_train.shape[0]
    W_init = np.random.randn(d, C)
    W = softmax_regression(X_train, y_train, W_init)[-1]

    # Dự đoán và đánh giá
    y_pred = pred(W, X_test)
    evaluate_model(y_test, y_pred)

In [39]:
pca_files = [
    ("../../data/dimension_reduction/pca/train_80.csv", "../../data/dimension_reduction/pca/test_20.csv", "8:2"),
    ("../../data/dimension_reduction/pca/train_70.csv", "../../data/dimension_reduction/pca/test_30.csv", "7:3"),
    ("../../data/dimension_reduction/pca/train_60.csv", "../../data/dimension_reduction/pca/test_40.csv", "6:4"),
]

for train_file, test_file, label in pca_files:
    train_test_from_file(train_file, test_file, label)



--- Tỉ lệ train:test = 8:2 ---
              precision    recall  f1-score   support

         0.0       0.98      0.74      0.84       332
         1.0       0.37      0.83      0.51        59
         2.0       0.36      0.46      0.41        35

    accuracy                           0.73       426
   macro avg       0.57      0.68      0.59       426
weighted avg       0.85      0.73      0.76       426

Confusion Matrix:
[[245  67  20]
 [  2  49   8]
 [  3  16  16]]
AUC (macro average): 0.7791

--- Tỉ lệ train:test = 7:3 ---
              precision    recall  f1-score   support

         0.0       0.98      0.76      0.85       497
         1.0       0.41      0.83      0.54        88
         2.0       0.41      0.57      0.48        53

    accuracy                           0.75       638
   macro avg       0.60      0.72      0.63       638
weighted avg       0.85      0.75      0.78       638

Confusion Matrix:
[[377  84  36]
 [  8  73   7]
 [  0  23  30]]
AUC (macro average

In [40]:
lda_files = [
    ("../../data/dimension_reduction/lda/train_80.csv", "../../data/dimension_reduction/lda/test_20.csv", "8:2"),
    ("../../data/dimension_reduction/lda/train_70.csv", "../../data/dimension_reduction/lda/test_30.csv", "7:3"),
    ("../../data/dimension_reduction/lda/train_60.csv", "../../data/dimension_reduction/lda/test_40.csv", "6:4")
]

for train_file, test_file, label in lda_files:
    train_test_from_file(train_file, test_file, label)



--- Tỉ lệ train:test = 8:2 ---
              precision    recall  f1-score   support

         0.0       0.96      0.79      0.87       332
         1.0       0.42      0.32      0.37        59
         2.0       0.31      0.97      0.48        35

    accuracy                           0.74       426
   macro avg       0.57      0.69      0.57       426
weighted avg       0.83      0.74      0.76       426

Confusion Matrix:
[[262  25  45]
 [ 11  19  29]
 [  0   1  34]]
AUC (macro average): 0.7842

--- Tỉ lệ train:test = 7:3 ---
              precision    recall  f1-score   support

         0.0       0.97      0.78      0.87       497
         1.0       0.43      0.83      0.56        88
         2.0       0.43      0.57      0.49        53

    accuracy                           0.77       638
   macro avg       0.61      0.72      0.64       638
weighted avg       0.85      0.77      0.79       638

Confusion Matrix:
[[387  75  35]
 [ 10  73   5]
 [  0  23  30]]
AUC (macro average